## 대회 소개

- 출처 : 캐글
  - 문제 제공자 : IEEE Computational Intelligence Society
- 문제 개요: 카드 기록을 활용한 사기 거래 탐지
- 대부분 컬럼이 비식별화되어 있어, 이전 문제와 다르게 데이터 탐색을 통한 데이터에 대한 이해가 매우 중요

## 사용 데이터
- train.csv: 모델 학습용 데이터(데이터가 커서 샘플링된 데이터 제공)/ test.csv: 모델 평가용 데이터
  - TransactionID: 거래 ID(비식별화)
  - TransactionDT: 거래 시각(비식별화)
  - TransactionAmt: 거래 금액(US Dollar)
  - ProductCD: 상품 코드
  - card1 - card6: 카드 관련 정보(비식별화)
  - P_emaildomain, R_emaildomain: 이메일 정보
  - M1 - M9: 기존 거래와의 매칭 정보
  - isFraud: 사기 거래 여부

## 사용 데이터의 특징
- 비식별화된 특징이 매우 많음
- 다수 결측이 포함되어 있음
- 클래스 불균형 문제 존재

## 데이터 분리
- df: sampled_train_transaction.csv
- 'TransactionID', 'TransactionDT' 변수는 삭제
- X: df에서 isFraud가 제거된 데이터 프레임
- Y: df에서 isFraud값만 가져온 Series

In [1]:
import os
import pandas as pd
import numpy as np
os.chdir(r'C:\Users\신상훈\Desktop\preprocessing\part-6.-실전-머신러닝-프로젝트\6. 실전 머신러닝 프로젝트\25. IEEE-CIS Fraud Detection\데이터')

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["font.size"] = 15
plt.rcParams["figure.figsize"] = (20, 5)

import warnings
warnings.filterwarnings('ignore')